In [ ]:
# extract sanger male normal lines from tracker

from depmapomics.tracker import getTracker
tracker = getTracker()
sanger = tracker[(tracker['source'] == 'SANGER') & (tracker['primary_disease'].str.lower()=='normal')]
assert (sanger['datatype'] == 'wes').all()

sanger['sex'].value_counts()

In [ ]:
sanger = sanger[sanger.sex == 'Male']
sanger

In [ ]:
sanger = sanger[['internal_bam_filepath', 'internal_bai_filepath', 'participant_id']]
sanger = sanger.rename(columns={'internal_bam_filepath': 'hg38_analysis_ready_bam', 'internal_bai_filepath': 'hg38_analysis_ready_bam_index'})
sanger

In [ ]:
# import the above lines to terra workspace broad-firecloud-ccle/DepMap_WES_PONS_hg38-copy
from depmapomics import loading
import dalmatian as dm

refworkspace = 'broad-firecloud-ccle/DepMap_WES_PONS_hg38-copy'
samplesetname = 'SANGER_XY'
samples = sanger

samples.index.name = 'sample_id'
refwm = dm.WorkspaceManager(refworkspace).disable_hound()
refwm.upload_samples(samples)

#creating a sample set
refwm.update_sample_set(sample_set_id=samplesetname,
                      sample_ids=samples.index)


In [ ]:
from genepy import terra

submission_id = refwm.create_submission("CNV_Somatic_Panel_Workflow_Agilent_XY", samplesetname)
await terra.waitForSubmission(refworkspace, submission_id)

In [ ]:
# create a sanger sampleset in broad-firecloud-ccle/DepMap_WES_CN_hg38-sandbox

tracker = getTracker()

sangerwes = tracker[(tracker['source'] == 'SANGER') & (tracker['datatype']=='wes')]
sangerwes

In [ ]:
refworkspace = 'broad-firecloud-ccle/DepMap_WES_CN_hg38-sandbox'
samplesetname = 'all_sanger'
samples = sangerwes

samples.index.name = 'sample_id'
refwm = dm.WorkspaceManager(refworkspace).disable_hound()
refwm.update_sample_set(sample_set_id=samplesetname,
                      sample_ids=samples.index)

In [ ]:
all_pairs = refwm.get_pairs()
sanger_pairs = all_pairs[all_pairs.case_sample.isin(sangerwes.index)]
sanger_pairs.index

In [ ]:
refwm.update_pair_set(pair_set_id=samplesetname,
                      pair_ids=sanger_pairs.index)

In [ ]:
# hg38_analysis_ready_bam_index != internal_bai_filepath for some lines
# update those lines in workspace:
terra_wes = refwm.get_samples()
terra_wes_sanger = terra_wes[terra_wes.index.isin(sangerwes.index)]
terra_wes_sanger

In [ ]:
sum(terra_wes_sanger['internal_bai_filepath'] == terra_wes_sanger['hg38_analysis_ready_bam_index'])

In [ ]:
terra_wes_sanger['hg38_analysis_ready_bam_index'] = terra_wes_sanger['internal_bai_filepath']
sum(terra_wes_sanger['internal_bai_filepath'] == terra_wes_sanger['hg38_analysis_ready_bam_index'])

In [ ]:
terra_wes.update(terra_wes_sanger)
terra_wes

In [ ]:
refwm.update_sample_attributes(terra_wes)

In [ ]:
submission_id = refwm.create_submission("CNV_sample_XY_agilent", 'all_sanger', 'pair_set', expression='this.pairs')
await terra.waitForSubmission(refworkspace, submission_id)